Isha's VAR Code 2


In [19]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product

import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
path ='/content/drive/MyDrive/TimeSeriesProject/Data/finalData.csv'
df =pd.read_csv(path)
df.describe()

In [ ]:
def parser(s):
    return datetime.strptime(s, '%Y-%m-%d')

In [18]:
macro_data = pd.read_csv(path, parse_dates=['datetime'], index_col='datetime')
print(macro_data.shape)  # (4018, 33)

macro_data = macro_data.asfreq(pd.infer_freq(macro_data.index))
macro_data= macro_data[["tempmax","tempmin","temp","avgTemp","dew","humidity"]]
macro_data.head()
macro_data.tail()

(4018, 33)


,tempmax,tempmin,temp,avgTemp,dew,humidity
datetime,,,,,,
2023-04-27,18.8,3.4,10.8,11.10,-0.9,48.2
2023-04-28,15.9,8.7,12.0,12.30,1.1,48.4
2023-04-29,13.9,6.2,10.1,10.05,2.4,59.7
2023-04-30,8.2,4.6,6.6,6.40,1.9,72.5
2023-05-01,9.9,4.8,6.7,7.35,2.1,72.9


In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, dpi=120, figsize=(10,6))
for i, ax in enumerate(axes.flatten()):
    data = macro_data[macro_data.columns[i]]
    ax.plot(data, color='red', linewidth=1)
    # Decorations
    ax.set_title(macro_data.columns[i])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

In [ ]:
ad_fuller_result_1 = adfuller(macro_data['temp'])#.diff()[1:])

print('temp')
print(f'ADF Statistic: {ad_fuller_result_1[0]}')
print(f'p-value: {ad_fuller_result_1[1]}')

print('\n---------------------\n')

ad_fuller_result_2 = adfuller(macro_data['tempmax'])#.diff()[1:])

print('tempmax')
print(f'ADF Statistic: {ad_fuller_result_2[0]}')
print(f'p-value: {ad_fuller_result_2[1]}')

print('\n---------------------\n')

ad_fuller_result_3 = adfuller(macro_data['tempmin'])#.diff()[1:])

print('tempmin')
print(f'ADF Statistic: {ad_fuller_result_2[0]}')
print(f'p-value: {ad_fuller_result_2[1]}')

In [ ]:
print('temp causes tempmax?\n')
print('------------------')
granger_1 = grangercausalitytests(macro_data[['tempmax', 'temp']], 19)

print('\tempmax causes temp?\n')
print('------------------')
granger_2 = grangercausalitytests(macro_data[['temp', 'tempmax']], 19)

print('\tempmin causes temp?\n')
print('------------------')
granger_3 = grangercausalitytests(macro_data[['tempmin', 'temp']], 19)

In [ ]:
macro_data = macro_data[['tempmax','temp','tempmin']]
print(macro_data.shape)

In [ ]:
train_df=macro_data[:-14]
test_df=macro_data[-14:]
train_df.tail()
test_df

In [ ]:
print(test_df.shape)

In [ ]:
model = VAR(train_df)#diff()[1:]


In [ ]:
sorted_order=model.select_order(maxlags=20)
print(sorted_order.summary())

In [17]:
var_model = VARMAX(train_df, order=(19,0),enforce_stationarity= True)
fitted_model = var_model.fit(disp=False)
print(fitted_model.summary())

                                     Statespace Model Results                                    
Dep. Variable:     ['tempmax', 'temp', 'tempmin', 'dew']   No. Observations:                 4004
Model:                                           VAR(19)   Log Likelihood              -32908.704
                                             + intercept   AIC                          66453.409
Date:                                   Tue, 23 May 2023   BIC                          68455.234
Time:                                           00:50:39   HQIC                         67162.968
Sample:                                       05-01-2012                                         
                                            - 04-17-2023                                         
Covariance Type:                                     opg                                         
Ljung-Box (L1) (Q):     0.00, 0.00, 0.00, 0.00   Jarque-Bera (JB):   12.89, 2040.20, 3365.68, 92.90
Prob(Q):          

In [ ]:
n_forecast = 14
predict = fitted_model.get_prediction(start=len(macro_data),end=len(macro_data) + n_forecast-1)#start="2023-02-05",end='2023-15-05')

predictions=predict.predicted_mean

In [ ]:
predictions.columns=['tempmax_predicted','temp_predicted','tempmin_predicted']
predictions

In [ ]:
test_vs_pred=pd.concat([test_df,predictions],axis=1)

In [ ]:
test_vs_pred.plot(figsize=(10,5))

In [ ]:
from sklearn.metrics import mean_squared_error
import math 
from statistics import mean

rmse_tempmax=math.sqrt(mean_squared_error(predictions['tempmax_predicted'],test_df['tempmax']))
print('Mean value of tempmax is : {}. Root Mean Squared Error is :{}'.format(mean(test_df['tempmax']),rmse_tempmax))

rmse_temp=math.sqrt(mean_squared_error(predictions['temp_predicted'],test_df['temp']))
print('Mean value of temp is : {}. Root Mean Squared Error is :{}'.format(mean(test_df['temp']),rmse_temp))

rmse_tempmin=math.sqrt(mean_squared_error(predictions['tempmin_predicted'],test_df['tempmin']))
print('Mean value of tempmin is : {}. Root Mean Squared Error is :{}'.format(mean(test_df['tempmin']),rmse_tempmin))